# **PubMedBERT Text Encoding**

In this colab, I will be getting the text encodings for the queries using a PubMedBERT model.  These will be saved for future reference.

## **Setup**

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.0 MB/s 
     |████████████████████████████████| 6.8 MB 51.4 MB/s 
     |████████████████████████████████| 596 kB 68.9 MB/s 
     |████████████████████████████████| 895 kB 55.5 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install --upgrade python-terrier

     |████████████████████████████████| 97 kB 361 kB/s 
     |████████████████████████████████| 1.1 MB 31.2 MB/s 
     |████████████████████████████████| 69 kB 9.8 MB/s 
     |████████████████████████████████| 255 kB 65.4 MB/s 
     |████████████████████████████████| 41 kB 389 kB/s 
     |████████████████████████████████| 72 kB 1.7 MB/s 
     |████████████████████████████████| 294 kB 7.4 MB/s 
     |████████████████████████████████| 6.4 MB 47.7 MB/s 
     |████████████████████████████████| 1.8 MB 49.7 MB/s 
     |████████████████████████████████| 126 kB 50.1 MB/s 
     |████████████████████████████████| 291 kB 9.3 MB/s 
     |████████████████████████████████| 59 kB 8.1 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=c115a67a758f39a04e79d309f8ad17a83ec830cd5dae58307d8c47cb86a3bff7
  Stored in directory: /root/.cache/pip/wheels/a8/ec/0c/fd9fcb409ad158d1f62534f9b1924b406f420

In [ ]:
import pyterrier as pt

if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
import pandas as pd
import pickle

In [ ]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = cord19.get_topics('title')

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [15.6MB/s]


In [ ]:
!wget https://github.com/DavidONeill75101/level-4-project/blob/master/Datasets/CORD-19_Datasets/round5_docs.csv?raw=true
round5_docs = pd.read_csv('/content/round5_docs.csv?raw=true').drop(columns=['Unnamed: 0'])

## **Prepare for Encoding**

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",output_hidden_states=True)

model = model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSeque

The following function will encode the query concatenated to the document with a '\n' character in between.

In [ ]:
def text_to_embed(text):
  # Tokenize it with appropriate padding and truncation
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=500)

  # Move the IDs of the tokens over to the GPU
  input_ids = inputs['input_ids'].to(device)

  # Run the model on the data
  outputs = model(input_ids=input_ids)

  # Extract the embeddings
  with torch.no_grad():
    # Get the final layer of the neural network, and average the embedding for all the tokens
    # Some researchers use the vector just for the first or final token of the sentence
    # instead of an average. I don't think there is a definitive best approach.
    # You could stick to the mean for now.
    embed = outputs.hidden_states[-1].squeeze().mean(axis=0)

    # Return the embedding to the CPU and convert to a numpy array
    embed = embed.cpu().numpy()

  return embed

## **Get Query Embeddings**

In [ ]:
query_embeddings = {}
for i in range(1, 51):
  query_embeddings[str(i)] = text_to_embed(topics.iloc[i-1]['query'])
